## Chapter Exercises

### Multiple choice

1. Given the following datatype:

```haskell
data Weekday =
    Monday
  | Tuesday
  | Wednesday
  | Thursday
  | Friday
```
we can say:

- a) Weekday is a type with five data constructors  ✅
- b) Weekday is a tree with five branches
- c) Weekday is a product type
- d) Weekday takes five arguments

2. and with the same datatype definition in mind, what is the type of the following function, `f`?

```haskell
f Friday = "Miller Time"
```
- a) `f :: [Char]`
- b) `f :: String -> String`
- c) `f :: Weekday -> String` ✅
- d) `f :: Day -> Beer`

3. Types defined with the data keyword
- a) must have at least one argument
- b) must begin with a capital letter ✅
- c) must be polymorphic
- d) cannot be imported from modules

4. The function `g xs = xs !! (length xs - 1)`
- a) is recursive and may not terminate
- b) delivers the head of `xs`
- c) delivers the final element of `xs` ✅
- d) has the same type as `xs`

### Ciphers
In the Lists chapter, you wrote a Caesar cipher. Now, we want to
expand on that idea by writing a Vigenère cipher. A Vigenère cipher is another substitution cipher, based on a Caesar cipher, but it
uses a series of Caesar ciphers for polyalphabetic substitution. The
substitution for each letter in the plaintext is determined by a fixed
keyword.
So, for example, if you want to encode the message `“meet at dawn”` the first step is to pick a keyword that will determine which
Caesar cipher to use. We’ll use the keyword `“ALLY”` here. You repeat
the keyword for as many characters as there are in your original
message:
```
MEET AT DAWN
ALLY AL LYAL
```
Now the number of rightward shifts to make to encode each
character is set by the character of the keyword that lines up with it.
The `’A’` means a shift of `0`, so the initial M will remain M. But the `’L’`
for our second character sets a rightward shift of `11`, so `’E’` becomes
`’P’`. And so on, so “meet at dawn” encoded with the keyword `“ALLY”`
becomes `“MPPR AE OYWY”`.

In [ ]:
module Vingere where

import Data.Char


encrypt :: [(Char, Char)] -> String
encrypt =
    map (shift . codeToShift)
    where
        padding                = ord 'A'
        codeToShift (x, code)  = (x, ord code - padding)
        shift (char, shift)    = chr ((((ord char - padding ) + shift ) `mod` 26) + padding)


vingere :: String -> String -> String
vingere text key =
    restoreNonAlpha $ encrypt charAndCode
    where
        cycledKey             = cycle (toUpperString key)
        charAndCode           = zip clearedText cycledKey
        clearedText           = removeNonAlpha (toUpperString text)
        toUpperString         = map toUpper
        removeNonAlpha        = filter isAlpha
        restoreNonAlpha text' = foldl selector "" text
            where
                selector :: String -> Char -> String
                selector acc x = 
                    if not (isAlpha x)
                    then acc ++ [x]
                    else acc ++ [text' !! (length acc - getNonAlpha acc)]
                    where
                        getNonAlpha :: String -> Int
                        getNonAlpha str = 
                            sum $ map (\char -> if isAlpha char then 0 else 1) str

### As-patterns

***As-patterns*** in Haskell are a nifty way to be able to pattern match on
part of something and still refer to the entire original value. Some
examples:

```haskell
f :: Show a => (a, b) -> IO (a, b)
f t@(a, _) = do
    print a
    return t
```

Here we pattern-matched on a tuple so we could get at the first
value for printing, but used the `@` symbol to introduce a binding
named `t` in order to refer to the whole tuple rather than just a part.

```haskell
Prelude> f (1, 2)
1
(1,2)
```

We can use as-patterns with pattern matching on arbitrary data
constructors, which includes lists:

```haskell
doubleUp :: [a] -> [a]
doubleUp [] = []
doubleUp xs@(x:_) = x : xs

Prelude> doubleUp []
[]

Prelude> doubleUp [1]
[1,1]

Prelude> doubleUp [1, 2]
[1,1,2]

Prelude> doubleUp [1, 2, 3]
[1,1,2,3]
```

Use as-patterns in implementing the following functions:

1. This should return True if (and only if) all the values in the first list appear in the second list, though they need not be contiguous.
```haskell
isSubseqOf :: (Eq a)
    => [a]
    -> [a]
    -> Bool
```

In [3]:
import Data.List (elemIndex)

isSubseqOf :: (Eq a)
    => [a]
    -> [a]
    -> Bool

isSubseqOf [] _ = True
isSubseqOf xs@(x:xs') ys =
    case elemIndex x ys of
        Nothing -> False
        Just i -> isSubseqOf xs' (drop i ys)

In [4]:
isSubseqOf "blah" "halbwoot"

False

In [5]:
isSubseqOf "blah" "wootblah"

True

2. Split a sentence into words, then tuple each word with the capitalized form of each.
```haskell
capitalizeWords :: String -> [(String, String)]

Prelude> capitalizeWords "hello world"
[("hello", "Hello"), ("world", "World")]
```

In [10]:
import Data.Char ( toUpper, toLower )


capitalize :: String -> String
capitalize (x:xs) = toUpper x : map toLower xs

capitalizeWords :: String -> [(String, String)]
capitalizeWords text =
    zip (words text) (map capitalize (words text))

In [11]:
capitalizeWords "hello world"

[("hello","Hello"),("world","World")]

### Language exercises

1. Write a function that capitalizes a word.

```haskell
capitalizeWord :: String -> String
capitalizeWord = undefined
```

Example output.

```haskell
Prelude> capitalizeWord "Chortle"
"Chortle"
Prelude> capitalizeWord "chortle"
"Chortle"
```

In [12]:
capitalizeWord :: String -> String
capitalizeWord (x:xs) = toUpper x : map toLower xs

2. Write a function that capitalizes sentences in a paragraph. Recognize when a new sentence has begun by checking for periods. Reuse the `capitalizeWord` function.

In [24]:
splitOn :: Char -> String -> [String]
splitOn c text =
    case nextSubstring of
        [] -> []
        _  -> nextSubstring : splitOn c rest
    where
        nextSubstring = getNext text
        rest = drop (length (takeWhile (== c) text ++ nextSubstring)) text
        getNext = takeWhile (/= c) . dropWhile (== c)

In [49]:
import Data.List (intercalate)

capitalizeParagraph :: String -> String
capitalizeParagraph text = 
    intercalate ". " (map capitalizeSentence sentences) ++ "."
    where
        sentences = splitOn '.' text
        capitalizeSentence sentence =
            case words sentence of
                [] -> ""
                first:rest -> unwords $ capitalizeWord first : rest

In [ ]:
let s = "blah. woot ha. im going home."
capitalizeParagraph s

"Blah. Woot ha. Im going home."